In [3]:
import os
from unstructured.partition.pdf import partition_pdf
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import MultiVectorRetriever
from PIL import Image
from google import genai
from langchain.docstore import InMemoryDocstore
import ollama
from tqdm import tqdm

from langchain.text_splitter import RecursiveCharacterTextSplitter
from pdf2image import convert_from_path

from groq import Groq
import base64
import os


/home/linuxachin/Desktop/Codes/fundify-gemini/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [3]:
# import module


# Store Pdf with convert_from_path function
images = convert_from_path('../Zarnik Deck - PedalUp Demo Day_compressed.pdf')

summ = []
for i in range(len(images)):
  
      # Save pages as images in the pdf
    path = '../pdf_images/page'+ str(i) +'.jpg'
    images[i].save(path, 'JPEG')

    base64_image = encode_image(path)

    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What's in this image?"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        model="meta-llama/llama-4-scout-17b-16e-instruct",
    )

    print(chat_completion.choices[0].message.content)
    # response = ollama.chat(model='gemma3:latest', 
    #             messages=[{
    #                 'role': 'user', 
    #                 'content': "Detailed summary of this image, in terms of contents. if no content then just say no content, output should be just bullet points",
    #                 'images': [path]
    #             }],
    #             # options={"temperature":0.7}
    #             )
    #         # image_summaries.append(response.text)
    summ.append(chat_completion.choices[0].message.content)





The image is an advertisement for Zarnik, a company that provides hotel supplies. The ad features a serene mountainous landscape with a lake and a hotel room on a deck overlooking the view.

* **Company Information**
	+ Company name: Zarnik
	+ Tagline: "Get everything your hotel needs."
	+ Website: www.zarnik.com
* **Services Offered**
	+ Find all products in one place
	+ Conveniently order online
	+ Get the order door delivered
* **Event Information**
	+ Event name: Pitch Deck PedalUp 3.0
	+ Date: December 2023

The image effectively communicates the benefits of using Zarnik's services, including convenience, ease of use, and reliability. The use of a peaceful and natural setting also evokes feelings of relaxation and tranquility, which are often associated with hotels and hospitality. Overall, the ad aims to appeal to hotel owners and managers who are looking for a convenient and reliable way to source their supplies.
The image presents information about the founders of a company, fe

In [4]:
doc = ' '.join(summ)
with open("zarnik_extract.txt", "w", encoding="utf-8") as f:
    f.write(doc)

In [2]:
with open("zarnik_extract.txt", "r", encoding="utf-8") as f:
    doc = f.read()


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)
docs = text_splitter.create_documents([doc])

# 3. Set up embedding model (using Sentence Transformers)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 4. Store in ChromaDB
db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
)

/tmp/ipykernel_23826/3721846955.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [5]:
retriever = db.as_retriever(search_kwargs={'k': 10,})

In [10]:
query = "Give a score based on problem clarity (1-10)"

retrieved_docs = retriever.get_relevant_documents(query,)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
You are a helpful assistant. Given the following context retrieved from a document, answer the question below in JSON containing score, reasons and why not 10 with no extra text.

Context:
{context}

Question:
{query}

"""
response = ollama.chat(model='gemma3:latest', 
                messages=[{
                    'role': 'user', 
                    'content': prompt,
                }],
                # options={"temperature":0.7}
                )
# print(response.text)
print(response['message']['content'])

# Example


```json
{
  "score": 8,
  "reasons": [
    "The text clearly identifies the problem: hotel owners and managers struggle with supplier and logistics issues.",
    "The consequences of this problem (supply shortages, bad reviews, lost revenue) are explicitly stated, strengthening the clarity.",
    "The use of a slide format and a visual component (image on the right) enhances understanding.",
    "The inclusion of key metrics such as 'Active Customers', 'Annual Value', and 'Retention Rate' adds to the problem's contextualization.",
    "The breakdown of the problem into actionable areas is well-defined."
  ],
  "why_not_10": "While the description is clear and provides a good overview, it could benefit from quantifying the 'problem' more precisely – e.g., estimating the average revenue loss due to these issues. Additionally, more detail on the type of supplier/logistics problems would improve the clarity further."
}
```


In [10]:
retrieved_docs

[Document(metadata={}, page_content='Model" and a table detailing various key performance indicators (KPIs). The left side of the image outlines the business model, while the right side features a table with specific data.'),
 Document(metadata={}, page_content='In summary, the image effectively showcases the two co-founders of a company, providing a clear visual representation of their identities and roles within the organization. The image presents a comprehensive overview of a company\'s traction, featuring various metrics and statistics that highlight its growth and achievements. The title "Traction" is prominently displayed at the top left corner.\n\n**Key Metrics:**'),
 Document(metadata={}, page_content="The image effectively communicates the company's growth, financial stability, and partnerships, providing a clear and concise overview of its traction. The use of visual elements and key metrics makes it easy to quickly understand the company's achievements and progress. The ima